In [ ]:
!pip install requests pandas openpyxl


In [ ]:
import requests
import pandas as pd
from datetime import datetime
from IPython.display import display

history = pd.DataFrame(columns=["IP","AbuseIPDB_Status","Confidence_Score","Final_Status","Timestamp"])


In [ ]:
ABUSE_API_KEY = "YOUR_ABUSE_KEY"

def check_abuseipdb(ip):
    url = f"https://api.abuseipdb.com/api/v2/check?ipAddress={ip}&maxAgeInDays=90"
    headers = {"Key": ABUSE_API_KEY, "Accept": "application/json"}

    try:
        res = requests.get(url, headers=headers).json()
        score = res['data']['abuseConfidenceScore']
        status = "Malicious" if score > 50 else "Safe"
        return status, score
    except Exception as e:
        return "Safe", 0


In [ ]:
def check_multiple_ips(ip_list):
    global history
    results = []

    for ip in ip_list:
        status, score = check_abuseipdb(ip)
        final_status = status

        new_row = pd.DataFrame([{
            "IP": ip,
            "AbuseIPDB_Status": status,
            "Confidence_Score": score,
            "Final_Status": final_status,
            "Timestamp": datetime.now()
        }])
        history = pd.concat([history, new_row], ignore_index=True)
        results.append(new_row)

    return pd.concat(results, ignore_index=True)


In [ ]:
def display_colored(df):
    def color_status(val):
        if val == "Malicious":
            color = 'red'
        elif val == "Safe":
            color = 'green'
        else:
            color = 'black'
        return f'color: {color}; font-weight: bold'

    display(df.style.applymap(color_status, subset=['Final_Status']))


In [ ]:
ip_input = input("Enter IPs separated by comma: ")

ip_list = [ip.strip() for ip in ip_input.split(",") if ip.strip()]

results_df = check_multiple_ips(ip_list)
display_colored(results_df)


Enter IPs separated by comma: 172.3.3.3


/tmp/ipython-input-2552784596.py:16: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  history = pd.concat([history, new_row], ignore_index=True)
/tmp/ipython-input-4222048257.py:11: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  display(df.style.applymap(color_status, subset=['Final_Status']))


,IP,AbuseIPDB_Status,Confidence_Score,Final_Status,Timestamp
0,172.3.3.3,Safe,0,Safe,2025-08-30 08:42:59.410109


In [ ]:
'''uploaded_file = "sample_ips.csv"
df_csv = pd.read_csv(uploaded_file)
ip_list = df_csv["IP"].tolist()

results_df = check_multiple_ips(ip_list)
display_colored(results_df)'''


/tmp/ipython-input-4222048257.py:11: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  display(df.style.applymap(color_status, subset=['Final_Status']))


,IP,AbuseIPDB_Status,Confidence_Score,Final_Status,Timestamp
0,8.8.8.8,Safe,0,Safe,2025-08-30 08:43:00.069357
1,192.168.1.38,Safe,0,Safe,2025-08-30 08:43:00.495708
2,8.8.8.8,Safe,0,Safe,2025-08-30 08:43:01.076407
3,1.1.1.1,Safe,0,Safe,2025-08-30 08:43:01.522281
4,8.8.8.8,Safe,0,Safe,2025-08-30 08:43:01.956480
5,1.0.2.4,Safe,0,Safe,2025-08-30 08:43:02.399490
6,8.8.8.8,Safe,0,Safe,2025-08-30 08:43:03.128613
7,8.8.4.4,Safe,0,Safe,2025-08-30 08:43:03.851608
8,1.1.1.1,Safe,0,Safe,2025-08-30 08:43:04.578993
9,9.9.9.9,Safe,0,Safe,2025-08-30 08:43:05.072873
